In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import dask
import gev_utils as gevu
from regridding import regrid_to_loca
from utils import loca_gard_mapping, gev_metric_ids

In [2]:
################
#### Paths #####
################
# Update these for reproduction

project_data_path = "/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024"
project_code_path = "/storage/home/dcl5300/work/current_projects/conus_comparison_lafferty-etal-2024"

In [3]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="30GiB",
    walltime="06:00:00",
)

cluster.scale(jobs=20)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.26:44545,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Fit GEV

In [6]:
%%time
# Stationary, L-moments
metric_id = "max_tasmax"

gevu.gev_fit_all(
    metric_id = metric_id,
    stationary = True,
    fit_method = "lmom",
    periods_for_level = [20, 50, 100, 200],
    levels_for_period = None,
    future_years = [2050,2100],
    hist_years = [1950,2014]
)

CPU times: user 6min 28s, sys: 35.2 s, total: 7min 3s
Wall time: 43min 49s


In [ ]:
%%time
# Stationary, L-moments
for metric_id in gev_metric_ids:
    gevu.gev_fit_all(
        metric_id = metric_id,
        stationary = True,
        fit_method = "lmom",
        periods_for_level = [20, 50, 100, 200],
        levels_for_period = None,
        future_years = [2050,2100],
        hist_years = [1950,2014]
    )

In [ ]:
client.shutdown()

## Regridding

In [11]:
%%time
# Regrid to LOCA using NN
method = "nearest"

for metric_id in metric_ids:
    # Save path
    store_path = f"{project_data_path}/extreme_value/loca_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/extreme_value/original_grids/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/extreme_value/original_grids/{metric_id}/GARD-LENS*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            # Regrid and store
            ds_tmp = xr.open_dataset(file)
            ds_out = regrid_to_loca(ds_tmp, method=method)
            ds_out.to_netcdf(f"{store_path}/{store_name}")

CPU times: user 28.6 s, sys: 6.82 s, total: 35.4 s
Wall time: 58.6 s
